# Свертки

На предыдущих занятиях мы познакомились с многослойным персептроном. Это полносвязная сеть, у которой все нейроны одного слоя связаны со всеми нейронами другого, нейроны разные.

Что произойдет, если мы удалим одну связь? Удалить связь, все равно что сделать нулевым ее вес. При разумных ограничениях, когда веса всех связей примерно одинаковы, это не сильно повлияет на результат – нейронов много, каждый вносит свой небольшой вклад, долю такого вклада можно посчитать.

Можно удалять связи с малым вкладом – это процедура *прореживания* нейронной сети, при этом мы экономим вычисления, ведь на 0 можно не умножать, и так ясен результат.

Можно ли обучать такую прореженную, или, как говорят, *неполносвязную* сеть?
Конечно! Теми же градиентным спуском и обратным распространением, но необходимо учитывать структуру связей.


Можем удалить одну связь - можем и больше. Вообще, можно изначально делать сети, в которых связи неполные, на них накладываются ограничения:
- некоторые связи отсутствуют
- число связей (входов) нейрона ограничено
- параметры связи (веса) могут быть одинаковыми у некоторых (или даже всех) нейронов в слое
- раз многих связей нет, можно ввести правила, по которым нейроны разных слоев соединяются между собой.  

Пример:
Сделаем одинаковые нейроны, у каждого по 4 входа, подключены как на [рисунке](https://towardsdatascience.com/derivation-of-convolutional-neural-network-from-fully-connected-network-step-by-step-b42ebafa5275), других связей нет.
В персептроне было бы 9 * 4=36 связей и весов, здесь 4 * 4=16 связей, но, так как нейроны мы взяли одинаковыми, то всего 4 разных веса.

![img](https://drive.google.com/uc?id=1RjwDbYPEO_OtnlP8UYhRfHN1L5MvEN2u)


Такой пример можно интерпретировать и по-другому. Давайте переформатируем вход и расположим входы в нейроны не в виде столбца-вектора, а в виде матрицы, ничего кроме расположения мы не меняем, это тот же самый слой входов!

![img](https://drive.google.com/uc?id=1KGOt0j-G-0RCicaQ6MDrbH7UHg4Zd_GW)

Давайте и веса (их 4 штуки) представим как матрицу 2 * 2. Подкрасим те входы, которые используются для конкретного нейрона (нейронов в нашем примере тоже 4, на рисунке сверху вниз).

![img](https://drive.google.com/uc?id=1Yc9QTJFahI1fBCyrtgCrtAw62JnG-_KY)


В таком представлении работы нейронов получится, что, как-бы, некое окошко, размером 2 * 2 скользит над входом - матрицей 3 * 3 -  принимает входы, над которыми оно висит, умножает их на коэффициенты, складывает такие произведения, возвращает выход (возможно добавляется смещение и применяется функция активации, но пока забудем про это).

Описанная операция называется (двумерной) **сверткой** массивов, набор коэффициентов – **ядром свертки**.

Сети, которые основаны на операции свертки – **сверточные сети**.



Свертки бывают разные, одномерные, двумерные, трехмерные и т.д.

Можно по-разному выполнять свертки, используя некоторые техники.

Все это определяет структуру связей *неполносвязного* слоя.

Проще всего понять свертки на двумерном примере, посмотрим на [некоторые](https://neurohive.io/ru/osnovy-data-science/glubokaya-svertochnaja-nejronnaja-set/).




## Двумерная свертка
В двумерной свертке ядро, т.е. веса - это двумерная матрица, определенной высоты и ширины. Ядро "скользит" над двумерной же матрицей входа (часто это изображение), поэлементно выполняя операцию умножения с той частью входных данных, над которой оно сейчас находится, и затем суммирует все полученные значения в один выходной элемент (пиксель).

Ядро повторяет эту процедуру с каждым положением, над которой оно "скользит", преобразуя двумерную матрицу в другую, все еще двумерную, матрицу выходов (признаков, по английски feature map). Признаки на выходе являются взвешенными суммами (где веса являются значениями самого ядра) признаков на входе. На рисунках показаны примеры свертки для ядра 3 * 3.

![img](https://drive.google.com/uc?id=1rJroGiRy0x90p6ZKASUoYDZue88R2ULi)

![img](https://drive.google.com/uc?id=1X00UjlA0tD9703F0Vi9gCSkccY3fCeho)


Размер ядра свертки определяет количество признаков, которые будут объединены для получения нового признака на выходе.

В примере, приведенном выше, мы имеем 5 * 5=25 признаков на входе и 3 * 3=9 признаков на выходе. Для полносвязного слоя (fully connected) мы бы имели весовую матрицу 25 * 9 = 225 параметров, а каждый выходной признак являлся бы взвешенной суммой **всех** признаков на входе. Свертка позволяет произвести такую операцию со всего 9-ю параметрами, ведь каждый признак на выходе получается не из каждого признака на входе, а только нескольких, находящегося в "примерно" том же месте.

# Техники сверток
Часто к сверткам применяют различные техники, которые помогают регулировать размер выхода и, конечно, результат.

## Набивка (padding)
Если вы заметили, на картинках выше выход имеет меньший размер чем вход, потому что ядро не может выйти за пределы входа. Это бывает вредно, ведь крайние элементы никогда не попадут в центр ядра.

Можно применить технику **набивки** (padding), добавить ко входу фиктивные элементы (нули, пунктир на рисунке) так, чтобы ядро могло выйти за пределы входа. Тогда и крайние элементы входа попадут в центр ядра, свернутся с ним. Размер набивки, число элементов которые мы добавляем, обычно выбирают так, чтобы размер выхода и размер входа (без набивки) совпадали. Но можно также немного регулировать размер выхода.  

![img](https://drive.google.com/uc?id=19NKX02jdr7SDtWxoDNmJcsFf-8wsN85S)


## Сдвиг (stride)
Но бывает и наоборот, надо сделать выход меньше входа, ведь не все элементы его одинаково полезны, зачем лишний раз считать похожие числа.

Тут применяют технику **сдвига** (stride), когда сдвигают окошко ядра над входом не на один элемент, а на несколько. На рисунке применен сдвиг на два элемента (по обеим осям), в результате чего вход 5 * 5 превратился в выход 2 * 2.

![img](https://drive.google.com/uc?id=1b1g3EHRPBsFBXEFXyzJv5hjpBJMDOjpc)

(Есть техники которые позволяют наоборот, увеличить размер выхода, но они более экзотические, не будем сейчас про них говорить).

# Многоканальные свертки, фильтры

Одна свертка, как и один нейрон, может не очень много, поэтому нужно использовать много сверток.

Для облегчения программирования и стандартизации принят такой подход (но отметим, что вполне можно пользоваться и другими):

А) Данные или входы представляются в виде *трехмерных массивов*, третьи измерения называются **каналами** (channel). Такое часто возникает, например, при работе с изображениями, которые можно представить в виде трех цветовых матриц RGB, имеем три канала.

Б) к матрице каждого канала входа применяется своя свертка, но все свертки с одинаковым размером ядра и свойствами (сдвиг, набивка и т.п.). На рисунке показана трехканальная свертка, для каждого цвета - своя.

![img](https://drive.google.com/uc?id=1ydMHFxkrfqZq8j_gcHW0Ob2nhjhABT-_)


    

В) Результаты сверток всех каналов объединяются: складываются, это можно сделать, ведь результирующие матрицы одного размера, а также добавляется смещение (bias, как и у персептрона).

![img](https://drive.google.com/uc?id=17xp36Rm5ZexkDVr45HbMJ5AahgURy3YS)

![img](https://drive.google.com/uc?id=1rhP-jZq1B6Jsf5t-uh6leiB99RKAlEd1)



Г) Такая свертка с несколькими каналами и смещением называется **фильтром** (filter), и в результате ее из трехмерного входного массива получается двумерный выходной.

Д) Даже многоканальной свертки (фильтра) может быть мало, поэтому таких фильтров (одинакового размера) применяют множество к одному и тому же входу. Множество фильтров составляют сверточный слой, в результате которого на выходе получается трехмерный массив, число каналов в котором равно числу фильтров.

# Сверточная нейронная сеть
Итак, стандартный пример сверточного слоя принимает трехмерный массив, возвращает трехмерный же, но с числом каналов равным числу фильтров в слое.

Сверточная нейронная сеть состоит из множества сверточных слоев, выходы одних являются входами для других. Но не совсем так, свертка сама по себе - линейная операция. Нам же нужны нелинейные, поэтому к результатам свертки применяют функции активации, обычно одинаковую для всего сверточного слоя.

В сверточной нейронной сети могут использоваться и другие слои, например:
- *полносвязные* слои, как в персептронах, часто они являются последними слоями.
- само собой, слои функций *активации*, часто это ReLU - кусочно-линейная функция.
- субдискретизирующие слои (*пулинг*, pooling), которые похожи на сверточные, но не имеют обучаемых параметров. Пример слоев maxpooling и average pooling показан на рисунке, когда из окошка выбирается максимальный элемент, или же находится среднее значение элементов окошка, которые и являются выходом. Такие слои помогают существенно уменьшать размер выходных массивов, а значит и число вычислений.
- слои [*dropout*](http://primo.ai/index.php?title=Dropout), когда в процессе обучения некоторые связи случайно обрываются. Это заставляет сеть обучаться так, чтобы меньше зависеть от конкретной связи (ведь она может в любой момент исчезнуть), а больше от их совокупности. Такой процесс сильно помогает в обобщении информации и является иллюстрацией пословицы "Тяжело в учении - легко в бою". Но конечно, он же и затрудняет обучение. Может применяться и в персептронных и в сверточных слоях.   

![img](https://drive.google.com/uc?id=1RFBuXqptVrvD-oin4JUgWOaDNYSicSD3)

![img](https://drive.google.com/uc?id=1Jd2hBWlaRnxaXv2lMmbl89KwCDFHCQxG)

# Keras
Итак, сверточные нейронные сети состоят из слоев сверток, полносвязных, активаций и др.
Так они и создаются на практике.

Один из самых удобных инструментов это библиотека [Keras](https://keras.io/) которая использует уже упомянутый нами фреймворк TensorFlow.
